In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from scipy.sparse import csr_matrix, hstack
from textblob import TextBlob

In [2]:
PATH = '../../data/'

train = pd.read_csv(PATH + 'nbsvm_one_files/' + 'toxic.csv')
test = pd.read_csv(PATH + 'cleaned_test.csv')

# label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
label_cols = ['toxic']

print('getting train')
train_sentence = train['comment_text_cleaned']

print('getting test')
test_sentence = test['comment_text_cleaned']

text = pd.concat([train_sentence, test_sentence])

print(train.shape)
print(test.shape)

getting train
getting test
(144276, 27)
(153164, 21)


In [3]:
# CountVectorizer(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None,\
#                 lowercase=True, preprocessor=None, tokenizer=None, stop_words=None,\
#                 token_pattern=’(?u)\b\w\w+\b’, ngram_range=(1, 1), analyzer=’word’, max_df=1.0,\
#                 min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class ‘numpy.int64’>)

# TfidfVectorizer(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None,\
#                 lowercase=True, preprocessor=None, tokenizer=None, analyzer=’word’, stop_words=None,\
#                 token_pattern=’(?u)\b\w\w+\b’, ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None,\
#                 vocabulary=None, binary=False, dtype=<class ‘numpy.int64’>, norm=’l2’, use_idf=True,\
#                 smooth_idf=True, sublinear_tf=False)

phrase_vectorizer = CountVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    token_pattern=r'\w{1,}') # another possible param would be binary=True
char_vectorizer = CountVectorizer(ngram_range=(1,5), 
                                  strip_accents='unicode', 
                                  max_features=200000, 
                                  analyzer='char')
phrase_tfidf = TfidfTransformer(sublinear_tf=True)
char_tfidf = TfidfTransformer(sublinear_tf=True)

print('fitting char')
char_vectorizer.fit(text.values)
print('fitting phrase')
phrase_vectorizer.fit(text.values)
print('transforming train char')
train_char_count = char_vectorizer.transform(train_sentence.values)
print('transforming train phrase')
train_phrase_count = phrase_vectorizer.transform(train_sentence.values)
print('transforming test char')
test_char_count = char_vectorizer.transform(test_sentence.values)
print('transforming test phrase')
test_phrase_count = phrase_vectorizer.transform(test_sentence.values)


print('fitting char')
train_char = char_tfidf.fit_transform(train_char_count)
print('fitting phrase')
train_phrase = phrase_tfidf.fit_transform(train_phrase_count)
print('transforming test char')
test_char = char_tfidf.transform(test_char_count)
print('transforming test phrase')
test_phrase = phrase_tfidf.transform(test_phrase_count)

train_tfidf = hstack((train_char, train_phrase), format='csr')
test_tfidf = hstack((test_char, test_phrase), format='csr')

train_count = hstack((train_char_count, train_phrase_count), format='csr')

train_tfidf

fitting char


KeyboardInterrupt: 

In [ ]:
l = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

def pr(y_i, y, train_features):
    p = train_features[y==y_i].sum(0)
    return (p + 1) / ((y == y_i).sum() + 1)

def get_nblogreg_model(label_cols, train_features, train, test_features):
    preds = np.zeros((test.shape[0], 6))
#     train_preds = np.zeros((train.shape[0], ))
    for i, j in enumerate(l):
        print(j)
        if j == label_cols[0]:
            print('fit', j)
            y = train[j].values
            r = np.log(pr(1, y, train_count) / pr(0, y, train_count))
            model = LogisticRegression()
            x_nb = train_features.multiply(r).tocsr()
            model.fit(x_nb, y)
            preds[:, i] = model.predict_proba(test_features.multiply(r))[:, 1]
#         train_preds[:, i] = model.predict_proba(x_nb)[:, 1]
#         print('accuracy is {}'.format(roc_auc_score(y, train_preds[:, i])))
    return preds

def save(model_name, y_test, label_cols, path, is_train=False):
    if is_train:
        submission = pd.read_csv(path + 'sample_train.csv')
        file_name = 'train_' + model_name
    else:
        submission = pd.read_csv(path + 'sample_submission.csv')
        file_name = label_cols[0]
    submission[l] = y_test
    submission.to_csv(path + model_name + '/' + file_name + '.csv', index=False)
    
print('done')

In [ ]:
print('predicting')
y_test = get_nblogreg_model(label_cols, train_tfidf, train, test_tfidf)
print('total score is {}'.format(roc_auc_score(train[label_cols], y_train)))

print('saving files')
model_name = 'nblogreg'
save(model_name, y_test, label_cols, PATH)
# save(model_name, y_train, label_cols, PATH, True)

print('done')